In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
from bs4 import BeautifulSoup
import json
from pprint import pprint
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
import highlight_text
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
from mplsoccer import Pitch, VerticalPitch
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
from mplsoccer import PyPizza, add_image, FontManager
from matplotlib.patches import Rectangle
import matplotlib.font_manager as font_manager
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
from mplsoccer import PyPizza, add_image, FontManager
import soccerdata as sd
import re
from unidecode import unidecode

pd.set_option("display.max_rows", None, "display.max_columns", None)
background = "black"
text_color = 'w'
primary = '#C8102E'
secondary = 'lightblue'
mpl.rcParams['xtick.color'] = text_color
mpl.rcParams['ytick.color'] = text_color

# setup a mplsoccer FontManager to download google fonts (Roboto-Regular / SigmarOne-Regular)
fm = FontManager()
fm_rubik = FontManager('https://raw.githubusercontent.com/google/fonts/main/ofl/rubikmonoone/'
                       'RubikMonoOne-Regular.ttf')

[09/02/24 19:22:20] INFO     No custom team name replacements found. You can configure these in       ]8;id=467286;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=231831;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py#90\90]8;;\
                             C:\Users\acer\soccerdata\config\teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=458621;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=382636;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py#196\196]8;;\
                             C:\Users\acer\soccerdata\config\league_dict.json.                                     

In [9]:
shots_df

,matchId


In [11]:
# Define the function to scrape shot data for a given matchId
def scrape_shots(mi):
    try: 
        params = {'matchId': mi}
        response = requests.get('https://www.fotmob.com/api/matchDetails', params=params)
        data = response.json()
        shotmap = data['content']['shotmap']['shots']
        shots_df = pd.DataFrame(shotmap)
        shots_df['matchId'] = mi
        return shots_df
    except Exception as e:
        print("Data not able to be scraped for MatchID : ",mi)

# Read the CSV containing the team names according to their respective Fotmob teamId
df_teamNameId = pd.read_csv("pl_team_name_and_id.csv")

# Define your list of matchIds (mi)
first_matchId = 4506263
last_matchId = 4506289
match_ids = [first_matchId + i for i in range(27)]  

# Initialize an empty list to store individual DataFrames
all_shots_dfs = []

# Loop through each matchId, scrape the shots data, merge with team names, and append to the list
for fotmob_matchId in match_ids:
    print(fotmob_matchId)
    shots_df = scrape_shots(fotmob_matchId)
    if shots_df is not None:
        shots_df = shots_df.merge(df_teamNameId[['teamId', 'teamName']], on='teamId', how='left')
    
        # Adding the opposition team name column
        team1 = shots_df['teamName'].unique()[0]
        team2 = shots_df['teamName'].unique()[1]
        
        def get_opposite_teamName(team):
            if team == team1:
                return team2
            elif team == team2:
                return team1
            else:
                return None
    
        # Apply the function to create the new column
        shots_df['oppositeTeam'] = shots_df['teamName'].apply(get_opposite_teamName)
        
        # Append the DataFrame to the list
        all_shots_dfs.append(shots_df)

# Concatenate all DataFrames in the list into a single DataFrame
final_shots_df = pd.concat(all_shots_dfs, ignore_index=True)

# Cross cheking if all the data has extracted succesfully or not
print(len(final_shots_df))

4506263
4506264
4506265
4506266
4506267
4506268
4506269
4506270
4506271
4506272
4506273
4506274
4506275
4506276
4506277
4506278
4506279
4506280
4506281
4506282
4506283
4506284
4506285
4506286
4506287
4506288
4506289
680


C:\Users\acer\AppData\Local\Temp\ipykernel_11332\3567602528.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_shots_df = pd.concat(all_shots_dfs, ignore_index=True)


In [12]:
final_shots_df.head()

,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,isOnTarget,blockedX,blockedY,goalCrossedY,goalCrossedZ,expectedGoals,expectedGoalsOnTarget,shotType,situation,period,isOwnGoal,onGoalShot,isSavedOffLine,isFromInsideBox,firstName,lastName,fullName,teamColor,matchId,teamName,oppositeTeam,shortName
0,2709319091,Miss,9879,493647,Adama Traoré,92.200000,18.534268,8,NaN,False,False,NaN,NaN,30.256000,4.584690,0.030144,0.000000,RightFoot,RegularPlay,FirstHalf,False,"{'x': 1.5530623084167812, 'y': 0.6772486772486...",False,True,Adama,Traoré,Adama Traoré,#000000,4506263,Fulham,Man Utd,NaN
1,2709323601,AttemptSaved,9879,469804,Kenny Tete,77.079126,34.076250,13,NaN,False,True,102.818966,33.61875,30.873750,2.029053,0.019572,0.284503,RightFoot,RegularPlay,FirstHalf,False,"{'x': 1.8270502645502642, 'y': 0.5367864078306...",False,False,Kenny,Tete,Kenny Tete,#000000,4506263,Fulham,Man Utd,NaN
2,2709324389,AttemptSaved,9879,1128945,Rodrigo Muniz,94.385965,30.645000,14,NaN,True,True,98.245614,32.47500,37.126250,0.359579,0.050167,0.000000,Header,FromCorner,FirstHalf,False,"{'x': 0.1729497354497358, 'y': 0.0951267051851...",False,True,Rodrigo,Muniz,Rodrigo Muniz,#000000,4506263,Fulham,Man Utd,NaN
3,2709330981,Miss,10260,422685,Bruno Fernandes,102.250000,46.032611,20,NaN,False,False,NaN,NaN,39.208810,0.179789,0.020179,0.000000,LeftFoot,RegularPlay,FirstHalf,False,"{'x': 0, 'y': 0.034516422982466426, 'zoomRatio...",False,True,Bruno,Fernandes,Bruno Fernandes,#C70101,4506263,Man Utd,Fulham,NaN
4,2709337569,Miss,10260,208494,Casemiro,79.331553,32.398750,27,NaN,False,False,NaN,NaN,38.452381,4.828000,0.042184,0.000000,RightFoot,RegularPlay,FirstHalf,False,"{'x': 0.3754413613904556, 'y': 0.6772486772486...",False,False,,Casemiro,Casemiro,#C70101,4506263,Man Utd,Fulham,NaN


In [13]:
ws = sd.WhoScored(leagues="ENG-Premier League", seasons="2024/2025")


first_match_id = 1821049
last_match_id = 1821079
match_ids = [first_match_id + i for i in range(31)]  

# Initialize an empty list to store individual DataFrames
all_events_dfs = []

for whoscoredid in match_ids:
    df_whoscored = ws.read_events(match_id=whoscoredid)
    all_events_dfs.append(df_whoscored)

# Concatenate all DataFrames in the list into a single DataFrame
events_df = pd.concat(all_events_dfs, ignore_index=True)

[09/02/24 20:17:29] INFO     Saving cached data to C:\Users\acer\soccerdata\data\WhoScored           ]8;id=660647;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=295117;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#263\263]8;;\

[09/02/24 20:17:44] INFO     Retrieving calendar for ENG-Premier League 2425                       ]8;id=468276;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=317586;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py#371\371]8;;\

[09/02/24 20:17:54] INFO     [1/10] Retrieving fixtures for ENG-Premier League 2425                ]8;id=43944;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=600093;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py#400\400]8;;\

[09/02/24 20:18:04] INFO     [2/10] Retrieving fixtures for ENG-Premier League 2425                ]8;id=701467;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=41868;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py#400\400]8;;\

[09/02/24 20:18:12] INFO     [3/10] Retrieving fixtures for ENG-Premier League 2425                ]8;id=429919;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=171933;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py#400\400]8;;\

...

[09/03/24 17:47:44] INFO     [10/10] Retrieving fixtures for ENG-Premier League 2425               ]8;id=654011;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=812597;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py#400\400]8;;\

C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py:424: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(all_schedules)


[09/03/24 17:47:53] INFO     [1/1] Retrieving game with id=1821079                                 ]8;id=541812;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=824381;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py#692\692]8;;\

[09/03/24 17:48:13] WARNING  No events found for game 1821079                                      ]8;id=635461;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=250719;file://C:\Users\acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\whoscored.py#765\765]8;;\

ValueError: No objects to concatenate